# Timing Data

In [1]:
from time_solvers import timed_trials, plot_solver_run_times
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import gillespy2
import os
if 'STOCHKIT_HOME' not in os.environ:
    os.environ['STOCHKIT_HOME'] = "/home/smatthe2/StochKit"

In [2]:
#Helper method for gathering imported items of a specific type:
def imported_classes(match_type):
    items = []
    key, value = None, None
    for key, value in globals().items():
        if isinstance(value, type) and issubclass(value, match_type) and value not in items:
            items.append(value)
    return items

In [3]:
# from example_models import *
# #Trichloroethylene, LacOperon, Schlogl, MichaelisMenten, ToggleSwitch, Example, Tyson2StateOscillator
# models = [MichaelisMenten(), Example(), Trichloroethylene()]#model() for model in imported_classes(gillespy2.Model)]
model_map = {}
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import tempfile
sbml_file = 'https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000035.2?filename=BIOMD0000000035_url.xml'
response = urlopen(sbml_file)
tmp = tempfile.NamedTemporaryFile(delete = False)
tmp.write(response.read())
tmp.close()
######

sbml_model, errors = gillespy2.import_SBML(tmp.name)
sbml_model.units = 'population'
sbml_model.timespan(np.linspace(0, 5, 101))
model_map[sbml_model] = tmp.name
print(os.linesep.join([error for error, code in errors]))
print("-----")

######
models = [sbml_model]
print('Imported models:')
for i, model in enumerate(models):
    print('{}:\t{}'.format(i+1, model.name))


-----
Imported models:
1:	Vilar2002_Oscillator


In [4]:
# from gillespy2.solvers.numpy import *
#BasicODESolver, BasicRootSolver, BasicTauLeapingSolver, NumPySSASolver, TauLeapingSolver

#CythonSSASolver
from gillespy2 import SSACSolver
#SSACSolver
from gillespy2.solvers.stochkit import *
#StochKitODESolver, StochKitSolver
solvers = imported_classes(gillespy2.GillesPySolver)
# solvers.remove(ODESolver)
solvers.remove(StochKitODESolver)
# solvers.remove(StochKitSolver)
# solvers.remove(SSACSolver)

#Here we pretend that roadrunner is a gillespy2 solver that can be precompiled for fair comparison
class RRSolver(SSACSolver):
    __name__ = 'RoadRunner'
    name = 'RoadRunner'
    model = None
    solver = None
    delete_directory = False
    results = []
    def __init__(self, sbml_model):
        try:
            import roadrunner
        except:
            raise Exception('Could not import Roadrunner')
        self.sbml_file = model_map[sbml_model]
        self.solver = roadrunner.RoadRunner(self.sbml_file)
        self.solver.setIntegrator('gillespie')
    @classmethod
    def run(self, model, t=20, number_of_trajectories=1, increment=0.05, resume=None, **kwargs):
        if self is None or self.model is None:
            self = RRSolver(model)
        for i in range(number_of_trajectories):
            self.results.append(self.solver.simulate())
            self.solver.reset()
        return self.results, 0
    

solvers.append(RRSolver) # Include Roadrunner in comparison
print('Imported solvers:')
for i, solver in enumerate(solvers):
    print('{}:\t{}'.format(i+1, solver.__name__))

Imported solvers:
1:	SSACSolver
2:	StochKitSolver
3:	RRSolver


In [ ]:
trajectories = [2**i for i in range(11)]
print(trajectories)
timing_data = timed_trials(models, solvers, trajectories, number_trials=30, output_file='timing_data.pkl')

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]


Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 1: 100%|██████████| 30/30 [00:00<00:00, 52.49it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 2: 100%|██████████| 30/30 [00:00<00:00, 32.72it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 4: 100%|██████████| 30/30 [00:01<00:00, 19.42it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 8: 100%|██████████| 30/30 [00:02<00:00, 10.42it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 16: 100%|██████████| 30/30 [00:05<00:00,  5.41it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 32: 100%|██████████| 30/30 [00:10<00:00,  2.76it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 64: 100%|██████████| 30/30 [00:20<00:00,  1.46it/s]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 128: 100%|██████████| 30/30 [00:41<00:00,  1.37s/it]
Model: Vilar2002_Oscillator, Solver: SSACSolver, Trajectories: 256: 100%|██████████

In [ ]:
#Ideally, we can run the timed tests once and save the results for later in a .pkl file
import pickle
with open('timing_data.pkl', 'rb') as f:
    timing_data = pickle.load(f)
    


In [ ]:
%matplotlib inline
styles = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'NumPySSASolver' : 'k*-',
    'BasicTauHybridSolver' : 'y*-',
    'BasicTauLeapingSolver' : 'm*-'
}
plot_solver_run_times(timing_data, line_styles=styles)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Log(seconds)', transformation=np.log)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Relative Speed to Stochkit', baseline_solver_name=StochKitSolver.name)

In [ ]:
import matplotlib.pyplot as plt
ylabel='Average run time (s)'
reduce=np.mean
line_styles=styles

In [ ]:
model_names = timing_data.keys()
for model in model_names:
    plt.figure(figsize=(10,8))
    plt.title('{} Timing Results'.format(model))
    plt.xlabel('Trajectories', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    baseline = None
    #if baseline_solver_name is not None:
    #    baseline = reduce(timing_data[model][baseline_solver_name][:, 1:], axis=1)
    #    if transformation is not None:
    #        baseline = transformation(times)
    solver_names = timing_data[model].keys()
    for solver in solver_names:
        trajectories = timing_data[model][solver][:,0]
        times = reduce(timing_data[model][solver][:, 1:], axis=1)
        #if transformation is not None:
        #    times = transformation(times)
        #if baseline is not None:
        #    times = np.divide(baseline, times)
        if solver in line_styles:
            plt.plot(trajectories, times, line_styles[solver], label=solver)
        else:
            plt.plot(trajectories, times, label=solver)
    plt.legend(loc='best', fontsize=16)


In [ ]:
ylabel='Average run time (s)'
reduce=np.mean
line_styles={
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'NumPySSASolver' : 'k*-',
    'BasicTauHybridSolver' : 'y*-',
    'BasicTauLeapingSolver' : 'm*-'
}
model_names = timing_data.keys()
for model in model_names:
    plt.figure(figsize=(10,8))
    plt.title('{} Timing Results'.format(model))
    plt.xlabel('Trajectories', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    baseline = None
    #if baseline_solver_name is not None:
    #    baseline = reduce(timing_data[model][baseline_solver_name][:, 1:], axis=1)
    #    if transformation is not None:
    #        baseline = transformation(times)
    #solver_names = timing_data[model].keys()
    solver_names = ['StochKitSolver', 'NumPySSASolver', 'SSACSolver']
    for solver in solver_names:
        trajectories = timing_data[model][solver][:,0]
        times = reduce(timing_data[model][solver][:, 1:], axis=1)
        #if transformation is not None:
        #    times = transformation(times)
        #if baseline is not None:
        #    times = np.divide(baseline, times)
        if solver in line_styles:
            plt.plot(trajectories, times, line_styles[solver], label=solver)
        else:
            plt.plot(trajectories, times, label=solver)
    plt.legend(loc='best', fontsize=16)


In [ ]:
solver1 = RRSolver(sbml_model)

In [ ]:
import roadrunner
rr = roadrunner.RoadRunner(tmp.name)

In [ ]:
rr.reset()
rr.setIntegrator('gillespie')
rr.simulate(0, 5)

In [ ]:
rr.plot()

In [ ]:
res.plot()

In [2]:
os.remove(tmp.name)

NameError: name 'tmp' is not defined